## Reading and visualizing the data

In the search for a solution ot keep all 50 million images in memory i evaluated:
- generating and compressing grasycale images 
- compressing the drawing



Both methods works: 
- Webp compression for grayscale images is very efficient with a fast decompression. The compression takes about 5h 30 min 
- Brotli compression of strokes in text format is as efficient as bz2 decompresses. Both compress much better than zlib. Brotli and zlib's decompression is 3 times fast compared to bz2. The compression takes about 10h 30 min (very slow compared to bz2 and zlib). 
- i have a macbook pro mid 2016

Decompression of 50 million images can be generated form the compressed state to images in 13 and 26 min from the compressed drawing and compressed images respectively


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
from code import *
from graphics import *

import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import gc
from pickle import *
import brotli
from core_ai.lib.data import *


In [20]:
data             = Path("../../data/google_quick_draw")
print(data.exists())
input_data       = data #Path("/Users/kasparlund/DropBox")

stroke_dir       = input_data/"train_simplified"
one_stroke_file  = stroke_dir/"ambulance.csv"

data_prepared    = data/"train"
data_prepared.mkdir(exist_ok=True)
data_compressed  = data/"compressed"
data_compressed.mkdir(exist_ok=True)

#astrokes_file = data_prepared/"train_simplified.csv"
vocab_file      = data_compressed/"vocab.json"

train_ds_file = data_prepared/"train_dataset.pkl"
valid_ds_file = data_prepared/"valid_dataset.pkl"

True


In [21]:
size, line_width  = 28, 1
valid_ratio = 0.1
linetype    = cv2.LINE_8 #cv2.LINE_AA
compressed_drawings = True

drawings_file    = data_prepared/f"c_drawings.pkl"
"""
images_file      = data_prepared/f"imgs_webp_{size}x{size}_lw{line_width}.pkl"
train_valid_name = "c_drawing_train_valid" if compressed_drawings else \
                    f"c_image_train_valid_{size}x{size}_lw{line_width}.gzip"
train_valid_file = data_prepared/f"{train_valid_name}.gzip"
"""


'\nimages_file      = data_prepared/f"imgs_webp_{size}x{size}_lw{line_width}.pkl"\ntrain_valid_name = "c_drawing_train_valid" if compressed_drawings else                     f"c_image_train_valid_{size}x{size}_lw{line_width}.gzip"\ntrain_valid_file = data_prepared/f"{train_valid_name}.gzip"\n'

In [22]:
print(data_compressed.exists()), print(data.exists()), print(one_stroke_file.exists())

True
True
True


(None, None, None)

## analyses one data tabel

## Prepare data

- compress all images or drawings into lists of compressed drawing or images, 
- create one annotation file
- replace space in labels with _
- create vocabulary file to tranlate between label and numeric tokens

- create a traiing and validation dataframe with the minimum number of required columns from the annotation file and the compressed image or drawing 

In [ ]:
from fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

import pickle
def merge_stroke_files(stroke_dir:Path, vocab_file:Path, data_compressed:Path, compressed_drawings:bool ):
    nrows = 0
    vocab = {}
    token = 0
    files = list( stroke_dir.glob("*.csv") )
    for f in  progress_bar(files):
        gc.collect()
        print(f"processing: {f.name}")
        df = pd.read_csv(f)
        #df.drop(["timestamp"], axis=1, inplace=True)
                
        #replace space in label with _, add label to vocab and create word_code column
        if sum(pd.isna(df['word'])) > 0: print(f"nan in word field for file:{f}")
        df["word"] = df["word"].apply( lambda word: word.strip().replace(" ", "_") )
        vocab[df["word"][0]] = token
        df["word_code"]      = token
        df["word_code"] = df["word_code"].astype(np.uint16)
        token += 1
                
        if compressed_drawings:
            df["drawing"] = [brotli.compress(txt.encode("utf-8"), quality=11) for txt in df["drawing"] ]
        else:    
            df["drawing"] = convert_strokes2webp(df["drawing"], size, line_width)
            
        #include header at first write
                  
        f_out = (data_compressed/f.name).with_suffix(".gzip")
        df.to_pickle(f_out, protocol=pickle.HIGHEST_PROTOCOL) 
                  
        #with (data_compressed/f.name).open("w") as fc: df.to_csv(fc, encoding="utf-8")                 
        nrows += df.shape[0]
        df = None
                                           
    with open(vocab_file, 'w') as fp:
        json.dump(vocab, fp)
                                           
    print(f"total number of records: {nrows}") 
    print(f"vocab size:{len(vocab)}:\n{vocab}")
    gc.collect()

if not vocab_file.exists():
    if compressed_drawings: 
        %time merge_stroke_files(stroke_dir, vocab_file, data_compressed, compressed_drawings=compressed_drawings)
    else:
        %time merge_stroke_files(stroke_dir, vocab_file, images_file,   compressed_drawings=False)        
        

processing: airplane.csv------------------------------------------------------------------------------| 0.00% [0/340 00:00<00:00]
processing: alarm clock.csv---------------------------------------------------------------------------| 0.29% [1/340 01:31<8:38:25]
processing: ambulance.csv-----------------------------------------------------------------------------| 0.59% [2/340 03:13<9:04:32]
processing: angel.csv---------------------------------------------------------------------------------| 0.88% [3/340 05:17<9:53:36]


In [ ]:
def read_and_split( file, columns:list,  valid_ratio:float):
    #df = pd.read_csv(f, encoding="utf-8", usecols=columns, dtype=dtype_columns)        
    df = pd.read_pickle(file)
    
    if columns is not None:
        drop_cols = [c for c in df.columns if not c in columns]
        df.drop(labels=drop_cols, axis="columns", inplace=True)
        
    ix_valid    = np.random.randint(0, len(df), size=int(len(df)*valid_ratio+.5) )
    df["train"] = True
    df.iloc[ix_valid, df.columns.get_loc("train")] = False
    
    return df

def read_and_split_all( compressed_dir:Path, columns:list=None, valid_ratio:float = valid_ratio ):
    total_rows = 0
    ext        = "*.gzip" #"*.csv"
    files      = list( compressed_dir.glob(ext) )
    #files     = list( compressed_dir.glob(ext) )[:1]
    for f in  progress_bar(files):
        print(f"processing: {f.name}")
        df = read_and_split( f, columns,  valid_ratio)
        yield df
                
        total_rows += df.shape[0]
    return df   

def draw_images_from_df(df, nb=1000, nrows=8, compressed_drawings=True):
    compressed = [df.drawing.iloc[i] for i in np.random.randint(0,len(df),nb) ]
    if compressed_drawings:
        print("reading and converting compressed drawings to images")
        ims = drawings2images(compressed, size, line_width, linetype, compressed_drawings) 
    else:    
        print("reading and decompressing images")
        ims = decode_images( compressed )
    
    print(f"plot images : {len(ims)}")
    plot_images( ims, cm.Greys, max_rows_cols=nrows )

In [ ]:
%%time
from core_ai.lib.data import *
#create a training file with numericalized words and training vs validation samples
def createDatasets():
    
    #move the vocab file to data_prepared
    with vocab_file.open('r') as fp:
        vocab = json.load(fp)        
        with (data_prepared/vocab_file.name).open('w') as fp:
            json.dump(vocab, fp)
            
    #read all to get the toal lenght        
    dfs=[]
    n_rows=0
    for df in read_and_split_all(data_compressed, columns=["word_code","drawing"], valid_ratio=valid_ratio):
        gc.collect()
        dfs.append(df)
        n_rows += len(df)
    
    #create datasets consisting of nd.arrays and fill them in
    gc.collect()
    train_ds = GDDataset( np.empty(n_rows,dtype=np.object), np.empty(n_rows,dtype=np.uint16))
    is_train = np.empty(n_rows,np.bool)
    
    n_rows = 0
    for df in dfs:
        train_ds.x[n_rows:n_rows+len(df)] = df.drawing.values[:]
        train_ds.y[n_rows:n_rows+len(df)] = df.word_code.values[:]
        is_train[n_rows:n_rows+len(df)]   = df.train.values[:]
        df.drop(labels=["train"], axis=1, inplace=True)
        n_rows += len(df)
    dfs = None
    gc.collect()
    
    
    valid_ds = GDDataset( train_ds.x[is_train==False], train_ds.y[is_train==False] )
    with valid_ds_file.open("wb") as f:
        %time pickle.dump(valid_ds, f, pickle.HIGHEST_PROTOCOL)
        valid_ds = None
        gc.collect()  
        
    train_ds.x = train_ds.x[is_train]
    train_ds.y = train_ds.y[is_train]
    is_train = None
    gc.collect()

    with train_ds_file.open("wb") as f:
        %time pickle.dump(train_ds, f, pickle.HIGHEST_PROTOCOL)
        train_ds=None
        gc.collect()  

createDatasets()

# in the training phase we would read data as shown below

In [ ]:
%%time
train_ds, valid_ds = readDatasets(train_ds_file, valid_ds_file)


In [ ]:
%%time
print(f"training dataset  : {len(train_ds.x), len(train_ds.x) }") 
print(f"validation dataset: {len(valid_ds.x), len(valid_ds.x) }") 
draw_images_from_dataset(train_ds, size, line_width, linetype, nb=1000, nrows=8, compressed_drawings=True), plt.show()


In [ ]:
#time to decompress 1000 images
nb, dataset = 1000, train_ds
compressed  = dataset.x[ np.random.randint(0,len(dataset.x),nb) ]
%time ims   = drawings2images(compressed, size, line_width, linetype, compressed_drawings) 